In [1]:
import json
task = 'connectivity'
filename = task+'_result_gpt4.json'
with open(filename, 'r', encoding='utf-8') as f:
    datas=json.load(f)

In [3]:
filename1 = task+'_test.json'
with open(filename1, 'r', encoding='utf-8') as f:
    gt=json.load(f)

In [4]:
print(len(datas))
print(len(gt))
print(gt[0])

300
300
{'input_prompt': 'Given an undirected graph as follows:\nNode 8 is connected to nodes 4, 5.\nNode 4 is connected to nodes 8, 1.\nNode 5 is connected to nodes 8, 2.\nNode 1 is connected to nodes 4, 7, 6.\nNode 3 is connected to nodes 10, 11.\nNode 10 is connected to nodes 3, 7.\nNode 11 is connected to nodes 3, 9.\nNode 7 is connected to nodes 10, 1.\nNode 0 is connected to node 9.\nNode 9 is connected to nodes 0, 11.\nNode 6 is connected to nodes 1, 2.\nNode 2 is connected to nodes 5, 6.\nIs there a path between node 0 and node 4?', 'answer': 'Yes'}


In [5]:
import re
import io
import sys


In [10]:
error_cnt = 0
correct_cnt = 0
import threading
import re

class TimeoutException(Exception):
    pass

def run_with_timeout(code, timeout=15):
    # 定义执行代码的函数
    def target():
        exec(code, globals(), globals())

    thread = threading.Thread(target=target)
    thread.start()
    thread.join(timeout)

    if thread.is_alive():
        raise TimeoutException("Code execution exceeded timeout limit.")

for i, (key,value) in enumerate(datas.items()):
    text = value['gpt4']
    code = re.sub(r'^```python\n|```$', '', text, flags=re.MULTILINE)
    ans = gt[i]['answer']

    # 创建一个 StringIO 对象来捕获输出
    output = io.StringIO()

    # 重定向 sys.stdout
    old_stdout = sys.stdout
    sys.stdout = output

    try:
        run_with_timeout(code,timeout=5)
    except TimeoutException as e:
        error_cnt = error_cnt + 1
        print(e)
        continue
    except:
        sys.stdout = old_stdout
        print(code)
        error_cnt = error_cnt + 1
        continue
    finally:
        sys.stdout = old_stdout
    

    # 获取输出内容
    output_content = output.getvalue()
    output_content = str(output_content).strip()
    # print("Captured Output:")
    
    if output_content == 'yes' or output_content == 'True':
        output_content = 'Yes'
    elif output_content == 'no' or output_content == 'False':                                 
        output_content = 'No'

    if output_content.strip() == str(ans).strip():
         correct_cnt = correct_cnt + 1
    else:
        #print(code)
        print(output_content,ans)
        # print(code)

Exception in thread Thread-945 (target):
Traceback (most recent call last):
  File "/home/qifanzhang/anaconda3/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/qifanzhang/anaconda3/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_2344740/2305305801.py", line 12, in target
  File "<string>", line 35, in <module>
  File "<string>", line 13, in has_path
KeyError: 8


 No


Exception in thread Thread-1195 (target):
Traceback (most recent call last):
  File "/home/qifanzhang/anaconda3/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/qifanzhang/anaconda3/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_2344740/2305305801.py", line 12, in target
  File "<string>", line 36, in <module>
  File "<string>", line 31, in has_path
KeyError: 11


 No


In [11]:
print(error_cnt)
print(correct_cnt)

0
298


In [12]:
print(correct_cnt/len(datas))
#max tokens是

0.9933333333333333


In [10]:
import re
output_content = "[1,3,5,2,4]"
pattern = r"[\{\[]([^}\]]*)[\}\]]"
match1 = re.search(pattern, output_content)


if match1:
    content_output = match1.group(1)
    
print(content_output)

1,3,5,2,4


In [24]:
import re

text = "{0, 1, 2, 3, 4, 10, 7}"
pattern = r"[\{\[]([^}\]]*)[\}\]]"
match = re.search(pattern, text)

if match:
    content = match.group(1)
    print(content)  # 输出: 0, 1, 2, 3, 4, 6, 7
    numbers = list(map(int, content.split(',')))
    numbers.sort()
    print(numbers)

0, 1, 2, 3, 4, 10, 7
[0, 1, 2, 3, 4, 7, 10]
